In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [2]:
df1 = pd.read_csv('Final_Data/Degree_Position_Count.csv')
df2 = pd.read_csv('Final_Data/Baseline_Degree_payback_grouped_mean.csv')
df3 = pd.read_csv('Final_Data/Weighted_Industry_Fields.csv')
df4 = pd.read_csv('Final_Data/Melted_State_Degree_Ratios.csv')

In [3]:
df4 = df4[df4['field'] != 'State_Total']
df4 = df4[['state', 'field', 'ratio']]

In [4]:
df1_grp = df1.groupby('category')['count'].sum()

In [5]:
df1_grp

category
Business, Admin, PR                 40168598
Science, Technology, Engineering    18381966
Name: count, dtype: int64

I have to discard this, as it doesn't include data for 3 of my 5 groupings

In [6]:

df2_grp = df2.groupby('field').mean()

In [7]:
df2_grp

,Starting Median Salary,Mid-Career Median Salary,Percent change from Starting to Mid-Career Salary
field,,,
"Architecture, Design",41775.000000,69675.000000,0.662500
"Business, Admin, PR",43828.571429,80942.857143,0.847143
"Consumer, Retail, Arts, Humanities",36420.000000,61030.000000,0.673300
"Health, Education, Government, Civil",40740.000000,62820.000000,0.562800
"Science, Technology, Engineering",48904.166667,82066.666667,0.687917


In [8]:
print(df3)

                                  field    count  normalize_industry_weight
0                  Architecture, Design   848572                  13.651310
1                   Business, Admin, PR  1995790                  14.506551
2    Consumer, Retail, Arts, Humanities  2363082                  14.675477
3  Health, Education, Government, Civil  1395824                  14.148995
4      Science, Technology, Engineering  2358322                  14.673461


The normalized weights here are to identify how many opportunities (roughly) are available in that field.

In [9]:
df4.groupby(['state','field']).mean()

ratio
state   field                                         
Alabama Architecture, Design                  0.111192
        Business, Admin, PR                   0.237358
        Consumer, Retail, Arts, Humanities    0.194429
        Health, Education, Government, Civil  0.159338
        Science, Technology, Engineering      0.297682
...                                                ...
Wyoming Architecture, Design                  0.104291
        Business, Admin, PR                   0.146799
        Consumer, Retail, Arts, Humanities    0.184515
        Health, Education, Government, Civil  0.198579
        Science, Technology, Engineering      0.365816

[255 rows x 1 columns]

In [10]:
df = df3[['field', 'normalize_industry_weight']]

In [11]:
df = df.merge(df2_grp, left_on='field', right_on='field', how='outer')

In [12]:
df

,field,normalize_industry_weight,Starting Median Salary,Mid-Career Median Salary,Percent change from Starting to Mid-Career Salary
0,"Architecture, Design",13.651310,41775.000000,69675.000000,0.662500
1,"Business, Admin, PR",14.506551,43828.571429,80942.857143,0.847143
2,"Consumer, Retail, Arts, Humanities",14.675477,36420.000000,61030.000000,0.673300
3,"Health, Education, Government, Civil",14.148995,40740.000000,62820.000000,0.562800
4,"Science, Technology, Engineering",14.673461,48904.166667,82066.666667,0.687917


In [13]:
df = df4.merge(df, left_on='field', right_on='field', how='outer')
df = df.dropna()

In [14]:
df = df[['state', 'field', 'ratio', 'normalize_industry_weight',
       'Starting Median Salary', 'Mid-Career Median Salary']]

In [15]:
df['score'] = df['ratio'] * df['normalize_industry_weight'] *  df['Starting Median Salary'] *  df['Mid-Career Median Salary']

df['score'] = np.log(df['score'])

In [16]:
df.corr()

,ratio,normalize_industry_weight,Starting Median Salary,Mid-Career Median Salary,score
ratio,1.000000,0.768744,0.515714,0.489210,0.948077
normalize_industry_weight,0.768744,1.000000,0.143817,0.260843,0.744866
Starting Median Salary,0.515714,0.143817,1.000000,0.891035,0.660553
Mid-Career Median Salary,0.489210,0.260843,0.891035,1.000000,0.694030
score,0.948077,0.744866,0.660553,0.694030,1.000000


In [17]:
df = df[['state','field', 'ratio','score']].sort_values('score', ascending = False)

In [18]:
df

,state,field,ratio,score
0,District of Columbia,"Science, Technology, Engineering",0.485053,24.075448
1,Washington,"Science, Technology, Engineering",0.421292,23.934516
2,California,"Science, Technology, Engineering",0.416775,23.923738
3,Maryland,"Science, Technology, Engineering",0.408777,23.904361
4,Massachusetts,"Science, Technology, Engineering",0.407389,23.900959
...,...,...,...,...
57,Virginia,"Architecture, Design",0.079575,21.874428
157,Massachusetts,"Health, Education, Government, Civil",0.084413,21.840607
155,California,"Health, Education, Government, Civil",0.061555,21.524806
51,District of Columbia,"Architecture, Design",0.051793,21.444991


In [19]:
df.groupby('field')['score'].mean().sort_values(ascending=False)

field
Science, Technology, Engineering        23.716886
Business, Admin, PR                     23.010448
Consumer, Retail, Arts, Humanities      22.707559
Health, Education, Government, Civil    22.275990
Architecture, Design                    22.124405
Name: score, dtype: float64

Despite a higher weight from availability, the Consumer/Retail etc field is outpaced by science, tech and business-oriented degrees and careers. 

In [20]:
df.to_csv('Final_Data/state_field_scores.csv')